In [ ]:
! pip install accelerate

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "google/gemma-1.1-2b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
)

In [12]:
def run_model(question: str) -> str:
  chat = [
      { "role": "user", "content": question },
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=3000)
  response = tokenizer.decode(outputs[0])
  return response

In [13]:
import pandas as pd

df = pd.read_csv("crt_dataset.csv")
questions = df["Question"].tolist()

In [15]:
for question in questions:
  print(run_model(question))
  print()

<bos><start_of_turn>user
Johnny’s mother had three children. The first child was named April. The second child was named May. What was the third child’s name probably?<end_of_turn>
<start_of_turn>model
The answer is Johnny.

The given information states that Johnny's mother had three children. The first child was named April and the second child was named May. Therefore, the third child's name is probably Johnny.<eos>

<bos><start_of_turn>user
The owner of X possesses four companies named Alpha, Beta, Gamma, and Delta. Can you guess the name of the fifth company?<end_of_turn>
<start_of_turn>model
The answer is the fifth company is Epsilon.

The owner possesses four companies named Alpha, Beta, Gamma, and Delta. Therefore, the fifth company must be Epsilon.<eos>

<bos><start_of_turn>user
During a marathon, if you manage to pass the runner in fourth place, what is your new position?<end_of_turn>
<start_of_turn>model
If you pass the runner in fourth place, you would now be in fourth place